In [16]:
import requests 
from bs4 import BeautifulSoup as soup
import pandas as pd 

In [17]:
web = 'https://cod-esports.fandom.com/wiki/Call_of_Duty_League/2021_Season/Stage_1'

In [18]:
response = requests.get(web)
response.status_code

200

In [19]:
s_object = soup(response.content,'html.parser')
s_object

<!DOCTYPE html>

<html class="client-nojs sse-other" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Call of Duty League 2021 - Stage 1 - Call of Duty Esports Wiki</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"01024bef74dd9ec9887f76cf7e562f21","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Call_of_Duty_League/2021_Season/Stage_1","wgTitle":"Call of Duty League/2021 Season/Stage 1","wgCurRevisionId":773848,"wgRevisionId":773848,"wgArticleId":129537,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["American Tournaments","Black Ops Cold War Tournaments","

The information we are looking for is a tr element that contains 4 different classes that go with it 
-  'standings-upwithbye teamhighlight teamhighlighter', = contains 2 teams (top 2  teams of the first stage qualifiers) 
-  'standings-up teamhighlight teamhighlighter',  = contains 4 teams (3 - 6 place  teams of the first stage qualifiers) 
-  'standings-stay teamhighlight teamhighlighter',  = contains 2 teams (7 - 8 place  teams of the first stage qualifiers) 
-  'standings-staydown teamhighlight teamhighlighter' = contains 4 teams (9 - 12 place  teams of the first stage qualifiers) 

In [5]:
# Define the search classes
search_classes = [
    'standings-upwithbye teamhighlight teamhighlighter',
    'standings-up teamhighlight teamhighlighter',
    'standings-stay teamhighlight teamhighlighter',
    'standings-staydown teamhighlight teamhighlighter'
]

Looping through all the teams that particiapted in stage 1 qualifiers 
- includes there results 
- map counts ( how many maps they won against out of a series)
- seriers counts ( how many times they bested there opponenets in a best of 3 series)

In [6]:
# Initialize lists to store the data
team_names = []
series_counts = []
map_counts = []

# Loop through each class and extract the data
for class_name in search_classes:
    target_elements = s_object.find_all('tr', class_=class_name)
    
    for team in target_elements:
        try:
            team_name = team.find('span', class_='teamname').text.strip()
        except:
            team_name = 'n/a'
        
        try:
            series_count = team.find_all('td')[2].text.strip()
        except:
            series_count = 'n/a'
        
        try:
            map_count = team.find_all('td')[4].text.strip()
        except:
            map_count = 'n/a'
        
        team_names.append(team_name)
        series_counts.append(series_count)
        map_counts.append(map_count)

In [7]:
# Create a DataFrame
data = {
    'Team Name': team_names,
    '(stage 1 / online - series_counts)': series_counts,
    '(stage 1 / online - map_counts)': map_counts
}
cdl_stage_teams = pd.DataFrame(data)
cdl_stage_teams

,Team Name,(stage 1 / online - series_counts),(stage 1 / online - map_counts)
0,Dallas Empire,4 - 1,14 - 7
1,Atlanta FaZe,5 - 0,15 - 6
2,Los Angeles Thieves,4 - 1,12 - 7
3,New York Subliners,3 - 2,11 - 7
4,OpTic Chicago,4 - 1,14 - 6
5,Los Angeles Guerrillas,2 - 3,8 - 13
6,Minnesota RØKKR,3 - 2,10 - 8
7,Paris Legion,2 - 3,10 - 11
8,Seattle Surge,1 - 4,6 - 14
9,London Royal Ravens,0 - 5,5 - 15


Now going to attempt to go through all five stages 
- looping through all five stages 
- Goal is to add up all stage series counts and online map counts
- to conlcude which teams played well online throughout the season online
- and then to find out there win percetages for both seires and map counts 

In [8]:
# Base URL
base_url = "https://cod-esports.fandom.com/wiki/Call_of_Duty_League/2021_Season/Stage_{}"

# List to store all data
all_team_data = []

# Set to keep track of team names
seen_team_names = set()

In [9]:
# Function to parse and sum counts
def parse_and_sum(count_string):
    counts = count_string.split(' - ')
    return int(counts[0]), int(counts[1])

In [10]:
def combine_counts(original_count, new_count):
    orig_wins, orig_losses = parse_and_sum(original_count)
    new_wins, new_losses = parse_and_sum(new_count)
    total_wins = orig_wins + new_wins
    total_losses = orig_losses + new_losses
    return f"{total_wins} - {total_losses}"

In [11]:
# Loop through each stage
for stage in range(1, 6):
    url = base_url.format(stage)
    response = requests.get(url)
    page_soup = soup(response.content, 'html.parser')
    
    # List of search parameters
    search_classes = [
        'standings-upwithbye teamhighlight teamhighlighter',
        'standings-up teamhighlight teamhighlighter',
        'standings-stay teamhighlight teamhighlighter',
        'standings-staydown teamhighlight teamhighlighter'
    ]
    
    # Loop through each search class to find all teams
    for class_name in search_classes:
        target_elements = page_soup.find_all('tr', class_=class_name)
        
        for results in target_elements:
            try:
                team_name = results.find('span', class_='teamname').text.strip()
            except:
                team_name = 'n/a'
            
            try:
                series_count = results.find_all('td')[2].text.strip()
            except:
                series_count = 'n/a'
            
            try:
                map_count = results.find_all('td')[4].text.strip()
            except:
                map_count = 'n/a'
            
            if team_name != 'n/a':
                if stage == 1:
                    # Add team names and their data during the first stage
                    seen_team_names.add(team_name)
                    all_team_data.append([team_name, series_count, map_count])
                else:
                    if team_name in seen_team_names:
                        # Update series and map counts for existing teams
                        for team_data in all_team_data:
                            if team_data[0] == team_name:
                                team_data[1] = combine_counts(team_data[1], series_count)
                                team_data[2] = combine_counts(team_data[2], map_count)
                                break


In [12]:
#Convert the list to a DataFrame
team_data_df = pd.DataFrame(all_team_data, columns=['Team Name', '(Stages/Online - Series Count)', '(Stages/Online - Map Count)'])
team_data_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Map Count)
0,Dallas Empire,17 - 8,60 - 41
1,Atlanta FaZe,22 - 3,72 - 24
2,Los Angeles Thieves,12 - 13,48 - 56
3,New York Subliners,16 - 9,57 - 44
4,OpTic Chicago,16 - 9,56 - 38
5,Los Angeles Guerrillas,5 - 20,31 - 66
6,Minnesota RØKKR,14 - 11,49 - 47
7,Paris Legion,8 - 17,39 - 58
8,Seattle Surge,5 - 20,31 - 65
9,London Royal Ravens,7 - 18,34 - 63


In [13]:
# Function to parse and sum counts from a series of count strings
def parse_and_sum_counts(count_series):
    total_wins = total_losses = 0
    for count in count_series:
        wins, losses = parse_and_sum(count)
        total_wins += wins
        total_losses += losses
    return total_wins, total_losses


In [14]:
# Group by team name and aggregate the counts
stages_df = team_data_df.groupby('Team Name').agg({
    '(Stages/Online - Series Count)': lambda x: f"{parse_and_sum_counts(x)[0]} - {parse_and_sum_counts(x)[1]}",
    '(Stages/Online - Map Count)': lambda x: f"{parse_and_sum_counts(x)[0]} - {parse_and_sum_counts(x)[1]}"
}).reset_index()


In [15]:
#Split 'Series Count' and 'Map Count' to get win and loss counts
series_win_loss_stages = stages_df['(Stages/Online - Series Count)'].str.split(' - ', expand=True).astype(int)
map_win_loss_stages = stages_df['(Stages/Online - Map Count)'].str.split(' - ', expand=True).astype(int)

In [16]:
# Calculate win percentages and round them
stages_df['(Stages/Online - Series Win Percentage)'] = (series_win_loss_stages[0] / (series_win_loss_stages[0] + series_win_loss_stages[1]) * 100).round(0)
stages_df['(Stages/Online - Map Win Percentage)'] = (map_win_loss_stages[0] / (map_win_loss_stages[0] + map_win_loss_stages[1]) * 100).round(0)

In [17]:
# Insert new columns next to existing ones
stages_df.insert(stages_df.columns.get_loc('(Stages/Online - Series Count)') + 1, '(Stages/Online - Series Win Percentage)', stages_df.pop('(Stages/Online - Series Win Percentage)'))
stages_df.insert(stages_df.columns.get_loc('(Stages/Online - Map Count)') + 1, '(Stages/Online - Map Win Percentage)', stages_df.pop('(Stages/Online - Map Win Percentage)'))
stages_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage)
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0
7,New York Subliners,16 - 9,64.0,57 - 44,56.0
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0
9,Paris Legion,8 - 17,32.0,39 - 58,40.0


Information on all the stages (ONLINE) throughout the year have been captured
- the amount of series wins , map wins through the online qualifiers
- the percentages of those wins
Now going to need the lan events to see if some teams perform better on LAN
- going to get winners bracket information, losers bracket, grand finals and then aggergate them
- need to figure out how to signal a series win 3 maps = 1 series win
- in the grand finals 5 win maps = 1 series win
- goal is to try to loop through all five majors and then agg both online and lan to get the final series counts for all teams and maps 

In [20]:
web = 'https://cod-esports.fandom.com/wiki/Call_of_Duty_League/2021_Season/Stage_1/Major'

In [21]:
response = requests.get(web)
response.status_code

200

In [22]:
s_object = soup(response.content,'html.parser')
s_object

<!DOCTYPE html>

<html class="client-nojs sse-other" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Call of Duty League 2021 - Stage 1 Major - Call of Duty Esports Wiki</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b27b4e7ed2df735695b36978291e4e5c","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Call_of_Duty_League/2021_Season/Stage_1/Major","wgTitle":"Call of Duty League/2021 Season/Stage 1/Major","wgCurRevisionId":787704,"wgRevisionId":787704,"wgArticleId":129539,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["American Tournaments","Black Ops Cold 

In [23]:
tourney_bracket = s_object.find('table',class_='wikitable md-table')
tourney_bracket

<table class="wikitable md-table" id="md-table" style="min-width:22em"><tbody><tr><th colspan="9"><div style="position:relative;"><div class="table-header-showhide">[<span class="sections-toggler mdv-showbuttons" data-toggler-hide=".mdv-showbuttons" data-toggler-show=".mdv-allweeks">show all</span><span class="sections-toggler mdv-allweeks toggle-section-hidden" data-toggler-hide=".mdv-allweeks" data-toggler-show=".mdv-showbuttons">hide all</span>]</div></div>VODs &amp; Match Links</th></tr><tr><th colspan="9"><div style="position:relative;"><div class="table-header-showhide">[<span class="sections-toggler mdv-showbuttons mdv-showbutton1" data-toggler-hide=".mdv-showbutton1" data-toggler-show=".mdv-week1">show</span><span class="sections-toggler mdv-allweeks mdv-week1 toggle-section-hidden" data-toggler-hide=".mdv-week1" data-toggler-show=".mdv-showbutton1">hide</span>]</div></div>Winners Bracket</th></tr><tr class="column-label-small mdv-allweeks mdv-week1 toggle-section-hidden"><th>T

In [24]:
#this includes all the teams in the winners bracket 
#and progress through as well and lost and was sent to the losers bracket
winners_bracket = tourney_bracket.find_all('tr')[3:8]
winners_bracket

[<tr class="mdv-allweeks mdv-week1 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a href="/wiki/Los_Angeles_Thieves" title="Los Angeles Thieves"><img alt="Los Angeles Thieveslogo std" class="lazyload" data-image-key="Los_Angeles_Thieveslogo_std.png" data-image-name="Los Angeles Thieveslogo std.png" data-relevant="0" data-src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/0/0e/Los_Angeles_Thieveslogo_std.png/revision/latest/scale-to-width-down/36?cb=20201106181825" decoding="async" height="15" loading="lazy" src="data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D" width="36"/></a>⁠</span><span class="teamname">LAT</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a href="/wiki/Los_Angeles_Guerrillas" title="Los Angeles Guerrillas"><img alt="Los Angeles Guerrillaslogo std" class="lazyload" data-image-key="Los_Angeles_Guerrillaslogo_std.pn

In [23]:
# Initialize lists
team_names = []
series_counts = []
map_counts = []

# Loop through the tr elements
for row in winners_bracket :
    columns = row.find_all('td')

    # Extract team names
    team1_name = columns[0].find('span', class_='teamname').text.strip()
    team2_name = columns[1].find('span', class_='teamname').text.strip()

    # Extract series result
    result = columns[2].text.strip()
    team1_result, team2_result = map(int, result.split(' - '))

    # Determine series counts based on the result
    if team1_result == 3:
        team1_series_count = '1 - 0'
        team2_series_count = '0 - 1'
    elif team2_result == 3:
        team1_series_count = '0 - 1'
        team2_series_count = '1 - 0'
    else:
        team1_series_count = team2_series_count = 'n/a'

    # Append data to lists
    team_names.extend([team1_name, team2_name])
    series_counts.extend([team1_series_count, team2_series_count])
    map_counts.extend([f'{team1_result} - {team2_result}', f'{team2_result} - {team1_result}'])


In [24]:
# Print results to verify
print('Team Names:', team_names)
print('Series Counts:', series_counts)
print('Map Counts:', map_counts)

Team Names: ['LAT', 'LAG', 'CHI', 'NY', 'ATL', 'LAG', 'DAL', 'CHI', 'DAL', 'ATL']
Series Counts: ['0 - 1', '1 - 0', '1 - 0', '0 - 1', '1 - 0', '0 - 1', '1 - 0', '0 - 1', '0 - 1', '1 - 0']
Map Counts: ['0 - 3', '3 - 0', '3 - 0', '0 - 3', '3 - 0', '0 - 3', '3 - 2', '2 - 3', '0 - 3', '3 - 0']


In [25]:
winners = pd.DataFrame({'Team Name':team_names, '(Major 1/Lan - Series Counts)': series_counts, '(Major 1/Lan - Map Counts)':map_counts
    
})
winners
#the issue with this is that they are not aggergated 

,Team Name,(Major 1/Lan - Series Counts),(Major 1/Lan - Map Counts)
0,LAT,0 - 1,0 - 3
1,LAG,1 - 0,3 - 0
2,CHI,1 - 0,3 - 0
3,NY,0 - 1,0 - 3
4,ATL,1 - 0,3 - 0
5,LAG,0 - 1,0 - 3
6,DAL,1 - 0,3 - 2
7,CHI,0 - 1,2 - 3
8,DAL,0 - 1,0 - 3
9,ATL,1 - 0,3 - 0


In [26]:
# Function to parse and sum counts from a string
def parse_and_sum(count_string):
    wins, losses = map(int, count_string.split(' - '))
    return wins, losses

In [27]:
# Function to combine counts
def combine_counts(count1, count2):
    wins1, losses1 = parse_and_sum(count1)
    wins2, losses2 = parse_and_sum(count2)
    total_wins = wins1 + wins2
    total_losses = losses1 + losses2
    return f"{total_wins} - {total_losses}"


In [28]:
# Initialize a dictionary to store the combined data
combined_data = {}

In [29]:
# Loop through the existing DataFrame and combine the counts
for index, row in winners.iterrows():
    team_name = row['Team Name']
    series_count = row['(Major 1/Lan - Series Counts)']
    map_count = row['(Major 1/Lan - Map Counts)']
    
    if team_name in combined_data:
        combined_data[team_name]['(Major 1/Lan - Series Counts)'] = combine_counts(combined_data[team_name]['(Major 1/Lan - Series Counts)'], series_count)
        combined_data[team_name]['(Major 1/Lan - Map Counts)'] = combine_counts(combined_data[team_name]['(Major 1/Lan - Map Counts)'], map_count)
    else:
        combined_data[team_name] = {
            '(Major 1/Lan - Series Counts)': series_count,
            '(Major 1/Lan - Map Counts)': map_count
        }

In [30]:
# Create a new DataFrame from the combined data
winnersb_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
winnersb_df.rename(columns={'index': 'Team Name'}, inplace=True)
winnersb_df
#winners bracket dataframe and results 

,Team Name,(Major 1/Lan - Series Counts),(Major 1/Lan - Map Counts)
0,LAT,0 - 1,0 - 3
1,LAG,1 - 1,3 - 3
2,CHI,1 - 1,5 - 3
3,NY,0 - 1,0 - 3
4,ATL,2 - 0,6 - 0
5,DAL,1 - 1,3 - 5


In [25]:
#now getting loser bracket results 
losers_bracket = tourney_bracket.find_all('tr')[10:20]
losers_bracket

[<tr class="mdv-allweeks mdv-week2 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a href="/wiki/Toronto_Ultra" title="Toronto Ultra"><img alt="Toronto Ultralogo std" class="lazyload" data-image-key="Toronto_Ultralogo_std.png" data-image-name="Toronto Ultralogo std.png" data-relevant="0" data-src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/5/52/Toronto_Ultralogo_std.png/revision/latest/scale-to-width-down/36?cb=20191026213309" decoding="async" height="15" loading="lazy" src="data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D" width="36"/></a>⁠</span><span class="teamname">TOR</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a href="/wiki/London_Royal_Ravens" title="London Royal Ravens"><img alt="London Royal Ravenslogo std" class="lazyload" data-image-key="London_Royal_Ravenslogo_std.png" data-image-name="London Royal Ravenslogo std.

In [32]:
# Initialize lists
team_names = []
series_counts = []
map_counts = []

# Loop through the tr elements
for row in losers_bracket:
    columns = row.find_all('td')

    # Extract team names
    team1_name = columns[0].find('span', class_='teamname').text.strip()
    team2_name = columns[1].find('span', class_='teamname').text.strip()

    # Extract series result
    result = columns[2].text.strip()
    team1_result, team2_result = map(int, result.split(' - '))

    # Determine series counts based on the result
    if team1_result == 3:
        team1_series_count = '1 - 0'
        team2_series_count = '0 - 1'
    elif team2_result == 3:
        team1_series_count = '0 - 1'
        team2_series_count = '1 - 0'
    else:
        team1_series_count = team2_series_count = 'n/a'

    # Append data to lists
    team_names.extend([team1_name, team2_name])
    series_counts.extend([team1_series_count, team2_series_count])
    map_counts.extend([f'{team1_result} - {team2_result}', f'{team2_result} - {team1_result}'])


In [33]:
losers = pd.DataFrame({'Team Name':team_names, '(Major 1/Lan - Series Counts)': series_counts, '(Major 1/Lan - Map Counts)':map_counts
    
})
losers

,Team Name,(Major 1/Lan - Series Counts),(Major 1/Lan - Map Counts)
0,TOR,1 - 0,3 - 2
1,LDN,0 - 1,2 - 3
2,SEA,0 - 1,1 - 3
3,FLA,1 - 0,3 - 1
4,PAR,0 - 1,1 - 3
5,FLA,1 - 0,3 - 1
6,MIN,0 - 1,2 - 3
7,TOR,1 - 0,3 - 2
8,NY,1 - 0,3 - 2
9,FLA,0 - 1,2 - 3


In [34]:
# Initialize a dictionary to store the combined data
combined_data = {}

In [35]:
# Loop through the existing DataFrame and combine the counts
for index, row in losers.iterrows():
    team_name = row['Team Name']
    series_count = row['(Major 1/Lan - Series Counts)']
    map_count = row['(Major 1/Lan - Map Counts)']
    
    if team_name in combined_data:
        combined_data[team_name]['(Major 1/Lan - Series Counts)'] = combine_counts(combined_data[team_name]['(Major 1/Lan - Series Counts)'], series_count)
        combined_data[team_name]['(Major 1/Lan - Map Counts)'] = combine_counts(combined_data[team_name]['(Major 1/Lan - Map Counts)'], map_count)
    else:
        combined_data[team_name] = {
            '(Major 1/Lan - Series Counts)': series_count,
            '(Major 1/Lan - Map Counts)': map_count
        }

In [36]:
# Create a new DataFrame from the combined data
losersb_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
losersb_df.rename(columns={'index': 'Team Name'}, inplace=True)
losersb_df
#losers bracket dataframe and results 

,Team Name,(Major 1/Lan - Series Counts),(Major 1/Lan - Map Counts)
0,TOR,2 - 1,6 - 7
1,LDN,0 - 1,2 - 3
2,SEA,0 - 1,1 - 3
3,FLA,2 - 1,8 - 5
4,PAR,0 - 1,1 - 3
5,MIN,0 - 1,2 - 3
6,NY,3 - 1,9 - 6
7,LAT,1 - 1,4 - 3
8,LAG,0 - 1,1 - 3
9,CHI,1 - 1,3 - 4


In [37]:
# Combine the two DataFrames
combined_df = pd.concat([winnersb_df, losersb_df])

In [38]:
# Initialize a dictionary to store the aggregated results
team_stats = {}

In [39]:
# Aggregate the results
for index, row in combined_df.iterrows():
    team_name = row['Team Name']
    series_wins, series_losses = parse_and_sum(row['(Major 1/Lan - Series Counts)'])
    map_wins, map_losses = parse_and_sum(row['(Major 1/Lan - Map Counts)'])
    
    if team_name not in team_stats:
        team_stats[team_name] = {'Series Wins': 0, 'Series Losses': 0, 'Map Wins': 0, 'Map Losses': 0}
    
    team_stats[team_name]['Series Wins'] += series_wins
    team_stats[team_name]['Series Losses'] += series_losses
    team_stats[team_name]['Map Wins'] += map_wins
    team_stats[team_name]['Map Losses'] += map_losses

In [40]:
# Create a new DataFrame from the aggregated results
aggregated_data = {
    'Team Name': [],
    '(Major 1/Lan - Series Counts)': [],
    '(Major 1/Lan - Map Counts)': []
}

In [41]:
for team, stats in team_stats.items():
    series_count = f"{stats['Series Wins']} - {stats['Series Losses']}"
    map_count = f"{stats['Map Wins']} - {stats['Map Losses']}"
    aggregated_data['Team Name'].append(team)
    aggregated_data['(Major 1/Lan - Series Counts)'].append(series_count)
    aggregated_data['(Major 1/Lan - Map Counts)'].append(map_count)

final_df = pd.DataFrame(aggregated_data)

# Display the final DataFrame in a table-like format
final_df

,Team Name,(Major 1/Lan - Series Counts),(Major 1/Lan - Map Counts)
0,LAT,1 - 2,4 - 6
1,LAG,1 - 2,4 - 6
2,CHI,2 - 2,8 - 7
3,NY,3 - 2,9 - 9
4,ATL,2 - 0,6 - 0
5,DAL,2 - 1,6 - 5
6,TOR,2 - 1,6 - 7
7,LDN,0 - 1,2 - 3
8,SEA,0 - 1,1 - 3
9,FLA,2 - 1,8 - 5


In [42]:
#now getting grand finals results 
grand_finals = tourney_bracket.find_all('tr')[22]
grand_finals

<tr class="mdv-allweeks mdv-week3 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe"><img alt="Atlanta FaZelogo std" data-image-key="Atlanta_FaZelogo_std.png" data-image-name="Atlanta FaZelogo std.png" data-relevant="0" decoding="async" height="15" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/5/5b/Atlanta_FaZelogo_std.png/revision/latest/scale-to-width-down/36?cb=20191026200905" width="36"/></a>⁠</span><span class="teamname">ATL</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Dallas_Empire" data-to-titles="Dallas Empire||Tooltip:Dallas Empire" href="/wiki/Dallas_Empire" title="Dallas Empire"><img alt="Dallas Empirelogo std" data-i

In [43]:
# Extract the team names
team_names = grand_finals.find_all('span', class_='teamname')
team1_name = team_names[0].text.strip()
team2_name = team_names[1].text.strip()

# Extract the series result
series_result = grand_finals.find_all('td')[2].text.strip()
team1_result, team2_result = map(int, series_result.split(' - '))

In [44]:
# Determine series counts based on the result
if team1_result == 5:
    team1_series_count = '1 - 0'
    team2_series_count = '0 - 1'
elif team2_result == 5:
    team1_series_count = '0 - 1'
    team2_series_count = '1 - 0'
else:
    team1_series_count = team2_series_count = 'n/a'

# Map counts
team1_map_count = f'{team1_result} - {team2_result}'
team2_map_count = f'{team2_result} - {team1_result}'

In [45]:
# Map counts
team1_map_count = f'{team1_result} - {team2_result}'
team2_map_count = f'{team2_result} - {team1_result}'

# Create DataFrame
data = {
    'Team Name': [team1_name, team2_name],
    '(Grand Final - Series Count)': [team1_series_count, team2_series_count],
    '(Grand Final - Map Count)': [team1_map_count, team2_map_count]
}

In [46]:
grand_final_df = pd.DataFrame(data)
grand_final_df 

,Team Name,(Grand Final - Series Count),(Grand Final - Map Count)
0,ATL,1 - 0,5 - 2
1,DAL,0 - 1,2 - 5


In [47]:
# Create dictionaries for series and map counts aggregation
combined_data = {}

In [48]:
# Combine the existing combined_df data
for i, row in combined_df.iterrows():
    team = row['Team Name']
    series_count = row['(Major 1/Lan - Series Counts)']
    map_count = row['(Major 1/Lan - Map Counts)']
    
    if team not in combined_data:
        combined_data[team] = {'Series Count': series_count, 'Map Count': map_count}
    else:
        combined_data[team]['Series Count'] = combine_counts(combined_data[team]['Series Count'], series_count)
        combined_data[team]['Map Count'] = combine_counts(combined_data[team]['Map Count'], map_count)

In [49]:
# Add the grand final data
for i, row in grand_final_df.iterrows():
    team = row['Team Name']
    series_count = row['(Grand Final - Series Count)']
    map_count = row['(Grand Final - Map Count)']
    
    if team not in combined_data:
        combined_data[team] = {'Series Count': series_count, 'Map Count': map_count}
    else:
        combined_data[team]['Series Count'] = combine_counts(combined_data[team]['Series Count'], series_count)
        combined_data[team]['Map Count'] = combine_counts(combined_data[team]['Map Count'], map_count)

In [50]:
# Create a new DataFrame from the combined data
Major1_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
Major1_df.columns = ['Team Name', 'Series Count', 'Map Count']

# Display the final combined DataFrame
Major1_df

#Fianl major 1 standings on lan

,Team Name,Series Count,Map Count
0,LAT,1 - 2,4 - 6
1,LAG,1 - 2,4 - 6
2,CHI,2 - 2,8 - 7
3,NY,3 - 2,9 - 9
4,ATL,3 - 0,11 - 2
5,DAL,2 - 2,8 - 10
6,TOR,2 - 1,6 - 7
7,LDN,0 - 1,2 - 3
8,SEA,0 - 1,1 - 3
9,FLA,2 - 1,8 - 5


Getting all major , on lan torunaments 
- looping through each major 

In [51]:
# Function to get team data from HTML
def get_team_data(tr_elements, is_grand_final=False):
    team_names = []
    series_counts = []
    map_counts = []
    
    for tr in tr_elements:
        tds = tr.find_all('td')
        team1_name = tds[0].find('span', class_='teamname').text.strip()
        team2_name = tds[1].find('span', class_='teamname').text.strip()
        team1_map_count = tds[2].text.strip()
        team1_wins, team1_losses = parse_and_sum(team1_map_count)
        
        team_names.append(team1_name)
        if (is_grand_final and team1_wins == 5) or (not is_grand_final and team1_wins == 3):
            series_counts.append("1 - 0")
            map_counts.append(f"{team1_wins} - {team1_losses}")
        else:
            series_counts.append("0 - 1")
            map_counts.append(f"{team1_losses} - {team1_wins}")
        
        team_names.append(team2_name)
        if (is_grand_final and team1_wins == 5) or (not is_grand_final and team1_wins == 3):
            series_counts.append("0 - 1")
            map_counts.append(f"{team1_losses} - {team1_wins}")
        else:
            series_counts.append("1 - 0")
            map_counts.append(f"{team1_wins} - {team1_losses}")

    return team_names, series_counts, map_counts

In [52]:
# Initialize the final DataFrame
all_data = []

# URLs for each stage
stages = [1, 2, 3, 4, 5]
base_url = 'https://cod-esports.fandom.com/wiki/Call_of_Duty_League/2021_Season/Stage_{}/Major'


In [53]:
for stage in stages:
    url = base_url.format(stage)
    response = requests.get(url)
    page_soup = soup(response.content, 'html.parser')
    
    tourney_bracket = page_soup.find('table', class_='wikitable md-table')
    
    # Winners bracket results
    winners_tr = tourney_bracket.find_all('tr')[3:8]
    winners_team_names, winners_series_counts, winners_map_counts = get_team_data(winners_tr)
    
    winners_data = {
        'Team Name': winners_team_names,
        'Series Counts': winners_series_counts,
        'Map Counts': winners_map_counts,
        'Stage': [f'Stage{stage}_Winners'] * len(winners_team_names)
    }
    
    # Losers bracket results
    losers_tr = tourney_bracket.find_all('tr')[10:20]
    losers_team_names, losers_series_counts, losers_map_counts = get_team_data(losers_tr)
    
    losers_data = {
        'Team Name': losers_team_names,
        'Series Counts': losers_series_counts,
        'Map Counts': losers_map_counts,
        'Stage': [f'Stage{stage}_Losers'] * len(losers_team_names)
    }
    
    # Grand finals results
    grand_final_tr = tourney_bracket.find_all('tr')[22:23]
    grand_final_team_names, grand_final_series_counts, grand_final_map_counts = get_team_data(grand_final_tr, is_grand_final=True)
    
    grand_final_data = {
        'Team Name': grand_final_team_names,
        'Series Counts': grand_final_series_counts,
        'Map Counts': grand_final_map_counts,
        'Stage': [f'Stage{stage}_GrandFinals'] * len(grand_final_team_names)
    }
    
    # Append all data to the final list
    all_data.extend(pd.DataFrame(winners_data).to_dict('records'))
    all_data.extend(pd.DataFrame(losers_data).to_dict('records'))
    all_data.extend(pd.DataFrame(grand_final_data).to_dict('records'))

In [54]:
# Convert the final list to a DataFrame
all_stages_df = pd.DataFrame(all_data)
all_stages_df

#includes all the results throughout all stage events 

,Team Name,Series Counts,Map Counts,Stage
0,LAT,0 - 1,3 - 0,Stage1_Winners
1,LAG,1 - 0,0 - 3,Stage1_Winners
2,CHI,1 - 0,3 - 0,Stage1_Winners
3,NY,0 - 1,0 - 3,Stage1_Winners
4,ATL,1 - 0,3 - 0,Stage1_Winners
...,...,...,...,...
155,CHI,1 - 0,1 - 3,Stage5_Losers
156,TOR,1 - 0,3 - 1,Stage5_Losers
157,CHI,0 - 1,1 - 3,Stage5_Losers
158,MIN,1 - 0,5 - 4,Stage5_GrandFinals


In [55]:
# Export the DataFrame to an Excel file
all_stages_df.to_excel('all_stages_data2021.xlsx', index=False)

# Read the Excel file
df = pd.read_excel('all_stages_data2021.xlsx')


In [56]:
#Function to adjust map counts based on series outcomes
def adjust_map_counts(row):
    series_wins, series_losses = map(int, row['Series Counts'].split(' - '))
    map_wins, map_losses = map(int, row['Map Counts'].split(' - '))
    
    if series_wins > series_losses:  # Team won the series
        if map_wins < map_losses:
            row['Map Counts'] = f"{map_losses} - {map_wins}"
    else:  # Team lost the series
        if map_wins > map_losses:
            row['Map Counts'] = f"{map_losses} - {map_wins}"
    return row

In [57]:
# Apply the function to each row in the DataFrame
df = df.apply(adjust_map_counts, axis=1)

# Export the modified DataFrame to an Excel file
df.to_excel('adjusted_file2021.xlsx', index=False)

In [58]:
# Load the Excel file
df = pd.read_excel('adjusted_file2021.xlsx')

In [59]:
# Function to parse series and map counts
def parse_counts(count_str):
    wins, losses = map(int, count_str.split(' - '))
    return wins, losses

In [60]:
# Apply parsing functions
df['Series Wins'], df['Series Losses'] = zip(*df['Series Counts'].apply(parse_counts))
df['Map Wins'], df['Map Losses'] = zip(*df['Map Counts'].apply(parse_counts))

In [61]:
# Aggregate data by team name
agg_df = df.groupby('Team Name').agg({
    'Series Wins': 'sum',
    'Series Losses': 'sum',
    'Map Wins': 'sum',
    'Map Losses': 'sum'
}).reset_index()

In [62]:
#Create new columns for aggregated counts
agg_df['(Stages/LAN - Series Counts)'] = agg_df['Series Wins'].astype(str) + ' - ' + agg_df['Series Losses'].astype(str)
agg_df['(Stages/LAN - Map Counts)'] = agg_df['Map Wins'].astype(str) + ' - ' + agg_df['Map Losses'].astype(str)

In [63]:
# Keep only the necessary columns
stages_eventdf = agg_df[['Team Name', '(Stages/LAN - Series Counts)', '(Stages/LAN - Map Counts)']]

# Display the DataFrame
stages_eventdf

,Team Name,(Stages/LAN - Series Counts),(Stages/LAN - Map Counts)
0,ATL,12 - 4,50 - 29
1,CHI,10 - 9,41 - 36
2,DAL,9 - 10,39 - 43
3,FLA,5 - 6,22 - 21
4,LAG,3 - 6,13 - 21
5,LAT,6 - 8,26 - 27
6,LDN,2 - 5,11 - 16
7,MIN,6 - 6,26 - 26
8,NY,8 - 8,28 - 37
9,PAR,0 - 5,3 - 15


In [64]:
# Calculating win percentages
series_win_loss_stages = stages_eventdf['(Stages/LAN - Series Counts)'].str.split(' - ', expand=True).astype(int)
map_win_loss_stages = stages_eventdf['(Stages/LAN - Map Counts)'].str.split(' - ', expand=True).astype(int)

In [65]:
# Insert new columns
stages_eventdf.insert(
    stages_eventdf.columns.get_loc('(Stages/LAN - Series Counts)') + 1, 
    '(Stages/LAN - Series Win Percentage)', 
    (series_win_loss_stages[0] / (series_win_loss_stages[0] + series_win_loss_stages[1]) * 100).round(0)
)

stages_eventdf.insert(
    stages_eventdf.columns.get_loc('(Stages/LAN - Map Counts)') + 1, 
    '(Stages/LAN - Map Win Percentage)', 
    (map_win_loss_stages[0] / (map_win_loss_stages[0] + map_win_loss_stages[1]) * 100).round(0)
)


Now have both lan events and online qualifiers 
- combining them both would tell us the entire story of the 2021 season
- stages_eventdf = LAN tournamnets, stages_df = online qualifires to determine seedings for lan tournamnets

In [66]:
stages_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage)
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0
7,New York Subliners,16 - 9,64.0,57 - 44,56.0
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0
9,Paris Legion,8 - 17,32.0,39 - 58,40.0


In [67]:
stages_eventdf

,Team Name,(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage)
0,ATL,12 - 4,75.0,50 - 29,63.0
1,CHI,10 - 9,53.0,41 - 36,53.0
2,DAL,9 - 10,47.0,39 - 43,48.0
3,FLA,5 - 6,45.0,22 - 21,51.0
4,LAG,3 - 6,33.0,13 - 21,38.0
5,LAT,6 - 8,43.0,26 - 27,49.0
6,LDN,2 - 5,29.0,11 - 16,41.0
7,MIN,6 - 6,50.0,26 - 26,50.0
8,NY,8 - 8,50.0,28 - 37,43.0
9,PAR,0 - 5,0.0,3 - 15,17.0


In [68]:
#changing team names to make it easier to combine 
# Dictionary to map team names from stages_eventdf to stages_df
team_name_mapping = {
    'ATL': 'Atlanta FaZe',
    'CHI': 'OpTic Chicago',
    'DAL': 'Dallas Empire',
    'FLA': 'Florida Mutineers',
    'LAG': 'Los Angeles Guerrillas',
    'LAT': 'Los Angeles Thieves',
    'LDN': 'London Royal Ravens',
    'MIN': 'Minnesota RØKKR',
    'NY': 'New York Subliners',
    'PAR': 'Paris Legion',
    'SEA': 'Seattle Surge',
    'TOR': 'Toronto Ultra'
}

# Replace the team names in stages_eventdf using .loc to avoid SettingWithCopyWarning
stages_eventdf.loc[:, 'Team Name'] = stages_eventdf['Team Name'].replace(team_name_mapping)


In [69]:
#now combining dataframes 
# Merge the two dataframes on 'Team Name'
entire2021_df = pd.merge(stages_df, stages_eventdf, on='Team Name', how='left')
# Display the combined dataframe
entire2021_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage),(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage)
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0,12 - 4,75.0,50 - 29,63.0
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0,9 - 10,47.0,39 - 43,48.0
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0,5 - 6,45.0,22 - 21,51.0
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0,2 - 5,29.0,11 - 16,41.0
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0,3 - 6,33.0,13 - 21,38.0
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0,6 - 8,43.0,26 - 27,49.0
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0,6 - 6,50.0,26 - 26,50.0
7,New York Subliners,16 - 9,64.0,57 - 44,56.0,8 - 8,50.0,28 - 37,43.0
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0,10 - 9,53.0,41 - 36,53.0
9,Paris Legion,8 - 17,32.0,39 - 58,40.0,0 - 5,0.0,3 - 15,17.0


Adding new columns 
- total seasons series counts and map count as well
- then will add the total series win and map percentages
- this will give us the entire output for the entire season 

In [70]:
# Split the series and map counts into wins and losses for both online and LAN
online_series_split = entire2021_df['(Stages/Online - Series Count)'].str.split(' - ', expand=True).astype(int)
lan_series_split = entire2021_df['(Stages/LAN - Series Counts)'].str.split(' - ', expand=True).astype(int)
online_map_split = entire2021_df['(Stages/Online - Map Count)'].str.split(' - ', expand=True).astype(int)
lan_map_split = entire2021_df['(Stages/LAN - Map Counts)'].str.split(' - ', expand=True).astype(int)


In [71]:
# Calculate the total series count by adding online and LAN series wins and losses
total_series_wins = online_series_split[0] + lan_series_split[0]
total_series_losses = online_series_split[1] + lan_series_split[1]
entire2021_df['Total - Series Count'] = total_series_wins.astype(str) + ' - ' + total_series_losses.astype(str)

In [72]:
# Calculate the total map count by adding online and LAN map wins and losses
total_map_wins = online_map_split[0] + lan_map_split[0]
total_map_losses = online_map_split[1] + lan_map_split[1]
entire2021_df['Total - Map Count'] = total_map_wins.astype(str) + ' - ' + total_map_losses.astype(str)

In [74]:
# Reorder the columns to place the new columns after the specified column
cols = entire2021_df.columns.tolist()
lan_map_win_percentage_index = cols.index('(Stages/LAN - Map Win Percentage)')
cols.insert(lan_map_win_percentage_index + 1, 'Total - Series Count')
cols.insert(lan_map_win_percentage_index + 2, 'Total - Map Count')
entire2021_df = entire2021_df[cols]

In [75]:
entire2021_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage),(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage),Total - Series Count,Total - Map Count,Total - Series Count,Total - Map Count
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0,12 - 4,75.0,50 - 29,63.0,34 - 7,122 - 53,34 - 7,122 - 53
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0,9 - 10,47.0,39 - 43,48.0,26 - 18,99 - 84,26 - 18,99 - 84
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0,5 - 6,45.0,22 - 21,51.0,17 - 19,69 - 70,17 - 19,69 - 70
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0,2 - 5,29.0,11 - 16,41.0,9 - 23,45 - 79,9 - 23,45 - 79
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0,3 - 6,33.0,13 - 21,38.0,8 - 26,44 - 87,8 - 26,44 - 87
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0,6 - 8,43.0,26 - 27,49.0,18 - 21,74 - 83,18 - 21,74 - 83
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0,6 - 6,50.0,26 - 26,50.0,20 - 17,75 - 73,20 - 17,75 - 73
7,New York Subliners,16 - 9,64.0,57 - 44,56.0,8 - 8,50.0,28 - 37,43.0,24 - 17,85 - 81,24 - 17,85 - 81
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0,10 - 9,53.0,41 - 36,53.0,26 - 18,97 - 74,26 - 18,97 - 74
9,Paris Legion,8 - 17,32.0,39 - 58,40.0,0 - 5,0.0,3 - 15,17.0,8 - 22,42 - 73,8 - 22,42 - 73


In [78]:
# Check for duplicate columns
duplicate_columns = entire2021_df.columns[entire2021_df.columns.duplicated()].tolist()

# Remove duplicate columns (keep the first occurrence)
entire2021_df = entire2021_df.loc[:, ~entire2021_df.columns.duplicated()]

In [80]:
# Ensure columns are strings
entire2021_df.loc[:, 'Total - Series Count'] = entire2021_df['Total - Series Count'].astype(str)
entire2021_df.loc[:, 'Total - Map Count'] = entire2021_df['Total - Map Count'].astype(str)

# Split the series and map counts into wins and losses
series_win_loss_stages = entire2021_df['Total - Series Count'].str.split(' - ', expand=True).astype(int)
map_win_loss_stages = entire2021_df['Total - Map Count'].str.split(' - ', expand=True).astype(int)

In [81]:
# Update existing columns with new values
entire2021_df['Total - Series Win Percentage'] = (series_win_loss_stages[0] / (series_win_loss_stages[0] + series_win_loss_stages[1]) * 100).round(0)
entire2021_df['Total - Map Win Percentage'] = (map_win_loss_stages[0] / (map_win_loss_stages[0] + map_win_loss_stages[1]) * 100).round(0)


C:\Users\Anthony\AppData\Local\Temp\ipykernel_2984\1644077667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entire2021_df['Total - Series Win Percentage'] = (series_win_loss_stages[0] / (series_win_loss_stages[0] + series_win_loss_stages[1]) * 100).round(0)


In [82]:
entire2021_df
#the entire 2021 cdl season 

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage),(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage),Total - Series Count,Total - Map Count,Total - Series Win Percentage,Total - Map Win Percentage
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0,12 - 4,75.0,50 - 29,63.0,34 - 7,122 - 53,83.0,70.0
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0,9 - 10,47.0,39 - 43,48.0,26 - 18,99 - 84,59.0,54.0
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0,5 - 6,45.0,22 - 21,51.0,17 - 19,69 - 70,47.0,50.0
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0,2 - 5,29.0,11 - 16,41.0,9 - 23,45 - 79,28.0,36.0
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0,3 - 6,33.0,13 - 21,38.0,8 - 26,44 - 87,24.0,34.0
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0,6 - 8,43.0,26 - 27,49.0,18 - 21,74 - 83,46.0,47.0
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0,6 - 6,50.0,26 - 26,50.0,20 - 17,75 - 73,54.0,51.0
7,New York Subliners,16 - 9,64.0,57 - 44,56.0,8 - 8,50.0,28 - 37,43.0,24 - 17,85 - 81,59.0,51.0
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0,10 - 9,53.0,41 - 36,53.0,26 - 18,97 - 74,59.0,57.0
9,Paris Legion,8 - 17,32.0,39 - 58,40.0,0 - 5,0.0,3 - 15,17.0,8 - 22,42 - 73,27.0,37.0


Finished the records for all cdl teams for the 2021 season 
- seperated there online records from there lan records
- can conclude which teams are better on LAN than Online and vice - versa
- also have final season records
  
Going to now get points distribution
- need points distribitopn
- champs placing
- rosters
- prize pool
- series counts and map counts as well for each team participating in the biggest tournament of the year 

In [26]:
web = 'https://cod-esports.fandom.com/wiki/Call_of_Duty_League/2021_Season'
response = requests.get(web)
s_object = soup(response.content, 'html.parser')
s_object

<!DOCTYPE html>

<html class="client-nojs sse-other" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Call of Duty League 2021 Season - Call of Duty Esports Wiki</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"5b742b658d14ead033f843140fbe5168","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Call_of_Duty_League/2021_Season","wgTitle":"Call of Duty League/2021 Season","wgCurRevisionId":778300,"wgRevisionId":778300,"wgArticleId":125926,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["International Tournaments","Black Ops Cold War Tournaments","Offline Tourna

In [84]:
points_elements = s_object.find_all('tr',class_='standings-upwithbye')
points_elements
#accessing top two teams 

[<tr class="standings-upwithbye teamhighlight teamhighlighter" data-teamhighlight="Atlanta FaZe"><td class="standings-place standings-upwithbye">1</td><td class="standings-team"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe"><img alt="Atlanta FaZelogo std" data-image-key="Atlanta_FaZelogo_std.png" data-image-name="Atlanta FaZelogo std.png" data-relevant="0" decoding="async" height="25" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/5/5b/Atlanta_FaZelogo_std.png/revision/latest?cb=20191026200905" width="60"/></a>⁠</span><span class="teamname"><a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe">Atlanta FaZe</a></span></span></td><td>34 - 7</td><td>83%</td><td>12

In [85]:
# Verify team_object content
print(f"Number of 'tr' elements found: {len(points_elements)}")
for i, team in enumerate(points_elements):
    print(f"Team {i+1} HTML: {team.prettify()}")

Number of 'tr' elements found: 4
Team 1 HTML: <tr class="standings-upwithbye teamhighlight teamhighlighter" data-teamhighlight="Atlanta FaZe">
 <td class="standings-place standings-upwithbye">
  1
 </td>
 <td class="standings-team">
  <span class="team">
   <span class="teamimage-right">
    ⁠
    <a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe">
     <img alt="Atlanta FaZelogo std" data-image-key="Atlanta_FaZelogo_std.png" data-image-name="Atlanta FaZelogo std.png" data-relevant="0" decoding="async" height="25" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/5/5b/Atlanta_FaZelogo_std.png/revision/latest?cb=20191026200905" width="60"/>
    </a>
    ⁠
   </span>
   <span class="teamname">
    <a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atl

In [86]:
points_distirubition = []

for point in points_elements:

    try:
        points = [td.text.strip() for td in point.find_all('td')[2:]]
        points_distirubition.append(points)

    except:
        points_distirubition.append('n/a')

In [87]:
points_distirubition

[['34 - 7', '83%', '123 - 53', '70%', '525'],
 ['28 - 17', '62%', '110 - 77', '59%', '415'],
 ['50', '75', '40', '60', '40', '75', '50', '75', '40', '20', '525'],
 ['10', '20', '20', '75', '50', '50', '40', '50', '40', '60', '415']]

In [88]:
#Points distribution
points_distribution = [['50', '75', '40', '60', '40', '75', '50', '75', '40', '20', '525'],
 ['10', '20', '20', '75', '50', '50', '40', '50', '40', '60', '415']]

# Accessing each team's points distribution
for i, team_points in enumerate(points_distribution):
    print(f"Team {i + 1} Points Distribution: {team_points}")

Team 1 Points Distribution: ['50', '75', '40', '60', '40', '75', '50', '75', '40', '20', '525']
Team 2 Points Distribution: ['10', '20', '20', '75', '50', '50', '40', '50', '40', '60', '415']


In [89]:
# Create a DataFrame
columns = ['Week ' + str(i) for i in range(1, len(points_distribution[0]) + 1)]
pt_distribution = pd.DataFrame(points_distribution, columns=columns)

# Add team names ()
team_names = ['Atlanta FaZe', 'Toronto Ultra']
pt_distribution.insert(0, 'Team Name', team_names)

pt_distribution 


,Team Name,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11
0,Atlanta FaZe,50,75,40,60,40,75,50,75,40,20,525
1,Toronto Ultra,10,20,20,75,50,50,40,50,40,60,415


In [90]:
# Function to extract points distribution
def extract_points_distribution(team_objects):
    points_distribution = []
    for team in team_objects:
        try:
            points = [td.text.strip() for td in team.find_all('td')[2:15]]
            points_distribution.append(points)
        except:
            points_distribution.append(['n/a'] * 13)
    return points_distribution

In [91]:
# Using the existing team_object
page_soup = s_object  # Assuming s_object contains the parsed HTML

In [92]:
# List of search parameters
search_params = [
    {'class_': 'standings-upwithbye', 'index': 2},
    {'class_': 'standings-up', 'index': 4},
    {'class_': 'standings-stay', 'index': 2},
    {'class_': 'standings-down', 'index': 4}
]

In [93]:
# Initialize the final points distribution list
final_points_distribution = []

In [94]:
# Loop through each search parameter and extract points distribution
for param in search_params:
    team_objects = page_soup.find_all('tr', class_=param['class_'])[param['index']:]
    points = extract_points_distribution(team_objects)
    final_points_distribution.extend(points)

In [95]:
# Create a DataFrame with the points distribution
columns = ['Week ' + str(i) for i in range(1, len(final_points_distribution[0]) + 1)]
pt_distribution = pd.DataFrame(final_points_distribution, columns=columns)

# Optional: Add team names (you need to extract or provide team names)
team_names = ['Team ' + str(i+1) for i in range(len(pt_distribution))]  # replace with actual team names
pt_distribution.insert(0, 'Team Name', team_names)

pt_distribution

,Team Name,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11
0,Team 1,50,75,40,60,40,75,50,75,40,20,525
1,Team 2,10,20,20,75,50,50,40,50,40,60,415
2,Team 3,40,60,30,50,30,20,30,60,40,40,400
3,Team 4,40,40,30,30,20,40,30,40,40,50,360
4,Team 5,30,50,40,20,40,60,40,30,10,20,340
5,Team 6,30,10,30,40,20,10,30,30,30,75,305
6,Team 7,40,30,10,30,30,30,10,0,30,30,240
7,Team 8,10,20,20,10,30,30,30,20,30,10,210
8,Team 9,10,0,20,20,0,0,10,20,10,30,120
9,Team 10,0,0,20,10,20,20,10,0,20,10,110


In [96]:
# New team names
new_team_names = [
    'Atlanta FaZe', 'Toronto Ultra', 'Dallas Empire', 'Optic Chicago',
    'New York Subliners', 'Minnesota RØKKR', 'Los Angeles Thieves', 'Florida Mutineers',
    'Seattle Surge', 'London Royal Ravens', 'Los Angeles Guerrillas', 'Paris Legion'
]

pt_distribution['Team Name'] = new_team_names
pt_distribution

,Team Name,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11
0,Atlanta FaZe,50,75,40,60,40,75,50,75,40,20,525
1,Toronto Ultra,10,20,20,75,50,50,40,50,40,60,415
2,Dallas Empire,40,60,30,50,30,20,30,60,40,40,400
3,Optic Chicago,40,40,30,30,20,40,30,40,40,50,360
4,New York Subliners,30,50,40,20,40,60,40,30,10,20,340
5,Minnesota RØKKR,30,10,30,40,20,10,30,30,30,75,305
6,Los Angeles Thieves,40,30,10,30,30,30,10,0,30,30,240
7,Florida Mutineers,10,20,20,10,30,30,30,20,30,10,210
8,Seattle Surge,10,0,20,20,0,0,10,20,10,30,120
9,London Royal Ravens,0,0,20,10,20,20,10,0,20,10,110


In [97]:
pt_distribution.columns = ['Team Name', 'Stage 1', 'Major 1', 'Stage 2', 'Major 2', 'Stage 3', 'Major 3', 'Stage 4', 'Major 4', 'Stage 5', 'Major 5', 'Total Season Points']
# Display the updated dataframe
pt_distribution

,Team Name,Stage 1,Major 1,Stage 2,Major 2,Stage 3,Major 3,Stage 4,Major 4,Stage 5,Major 5,Total Season Points
0,Atlanta FaZe,50,75,40,60,40,75,50,75,40,20,525
1,Toronto Ultra,10,20,20,75,50,50,40,50,40,60,415
2,Dallas Empire,40,60,30,50,30,20,30,60,40,40,400
3,Optic Chicago,40,40,30,30,20,40,30,40,40,50,360
4,New York Subliners,30,50,40,20,40,60,40,30,10,20,340
5,Minnesota RØKKR,30,10,30,40,20,10,30,30,30,75,305
6,Los Angeles Thieves,40,30,10,30,30,30,10,0,30,30,240
7,Florida Mutineers,10,20,20,10,30,30,30,20,30,10,210
8,Seattle Surge,10,0,20,20,0,0,10,20,10,30,120
9,London Royal Ravens,0,0,20,10,20,20,10,0,20,10,110


In [98]:
# Sample data
data = {
    "Team Name": ["Atlanta FaZe", "Toronto Ultra", "Dallas Empire", "Optic Chicago", "New York Subliners", 
                  "Minnesota RØKKR", "Los Angeles Thieves", "Florida Mutineers", "Seattle Surge", 
                  "London Royal Ravens", "Los Angeles Guerrillas", "Paris Legion"],
    "Stage 1": [50, 10, 40, 40, 30, 30, 40, 10, 10, 0, 20, 20],
    "Major 1": [75, 20, 60, 40, 50, 10, 30, 20, 0, 0, 30, 10],
    "Stage 2": [40, 20, 30, 30, 40, 30, 10, 20, 20, 20, 20, 20],
    "Major 2": [60, 75, 50, 30, 20, 40, 30, 10, 20, 10, 0, 0],
    "Stage 3": [40, 50, 30, 20, 40, 20, 30, 30, 0, 20, 10, 10],
    "Major 3": [75, 50, 20, 40, 60, 10, 30, 30, 0, 20, 10, 0],
    "Stage 4": [50, 40, 30, 30, 40, 30, 10, 30, 10, 10, 0, 20],
    "Major 4": [75, 50, 60, 40, 30, 30, 0, 20, 20, 0, 10, 10],
    "Stage 5": [40, 40, 40, 40, 10, 30, 30, 30, 10, 20, 0, 10],
    "Major 5": [20, 60, 40, 50, 20, 75, 30, 10, 30, 10, 0, 0],
    "Total Season Points": [525, 415, 400, 360, 340, 305, 240, 210, 120, 110, 100, 100]
}
pt_distribution = pd.DataFrame(data)

# Define placement columns
placement_columns = ['Major 1', 'Major 2', 'Major 3', 'Major 4', 'Major 5']

# Create new columns for each placement type
pt_distribution['Tournament Wins'] = (pt_distribution[placement_columns] == 75).sum(axis=1)
pt_distribution['2nd Place'] = (pt_distribution[placement_columns] == 60).sum(axis=1)
pt_distribution['3rd Place'] = (pt_distribution[placement_columns] == 50).sum(axis=1)
pt_distribution['4th Place'] = (pt_distribution[placement_columns] == 40).sum(axis=1)
pt_distribution['5th - 6th Place'] = (pt_distribution[placement_columns] == 30).sum(axis=1)
pt_distribution['7th - 8th Place'] = (pt_distribution[placement_columns] == 20).sum(axis=1)
pt_distribution['9th - 10th Place'] = (pt_distribution[placement_columns] == 10).sum(axis=1)
pt_distribution['11th - 12th Place'] = (pt_distribution[placement_columns] == 0).sum(axis=1)

pt_distribution

,Team Name,Stage 1,Major 1,Stage 2,Major 2,Stage 3,Major 3,Stage 4,Major 4,Stage 5,Major 5,Total Season Points,Tournament Wins,2nd Place,3rd Place,4th Place,5th - 6th Place,7th - 8th Place,9th - 10th Place,11th - 12th Place
0,Atlanta FaZe,50,75,40,60,40,75,50,75,40,20,525,3,1,0,0,0,1,0,0
1,Toronto Ultra,10,20,20,75,50,50,40,50,40,60,415,1,1,2,0,0,1,0,0
2,Dallas Empire,40,60,30,50,30,20,30,60,40,40,400,0,2,1,1,0,1,0,0
3,Optic Chicago,40,40,30,30,20,40,30,40,40,50,360,0,0,1,3,1,0,0,0
4,New York Subliners,30,50,40,20,40,60,40,30,10,20,340,0,1,1,0,1,2,0,0
5,Minnesota RØKKR,30,10,30,40,20,10,30,30,30,75,305,1,0,0,1,1,0,2,0
6,Los Angeles Thieves,40,30,10,30,30,30,10,0,30,30,240,0,0,0,0,4,0,0,1
7,Florida Mutineers,10,20,20,10,30,30,30,20,30,10,210,0,0,0,0,1,2,2,0
8,Seattle Surge,10,0,20,20,0,0,10,20,10,30,120,0,0,0,0,1,2,0,2
9,London Royal Ravens,0,0,20,10,20,20,10,0,20,10,110,0,0,0,0,0,1,2,2


Getting information on the bigest torunament of the year 
- cod champs placements
- earnings
- rosters
- map counts
- series counts


In [27]:
web = 'https://cod-esports.fandom.com/wiki/Call_of_Duty_League_Championship_2021'
response = requests.get(web)


In [28]:
s_object = soup(response.content, 'html.parser')
s_object

<!DOCTYPE html>

<html class="client-nojs sse-other" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Call of Duty League Championship 2021 - Call of Duty Esports Wiki</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6f798332781f85af162b49dfe76b15a8","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Call_of_Duty_League_Championship_2021","wgTitle":"Call of Duty League Championship 2021","wgCurRevisionId":744400,"wgRevisionId":744400,"wgArticleId":125927,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["American Tournaments","Black Ops Cold War Tournaments","P

In [29]:
champs_element = s_object.find('table',class_='wikitable2 tournament-results zebra-manual')
champs_element

<table class="wikitable2 tournament-results zebra-manual"><tbody><tr><th>Place</th><th>Prize</th><th>Prize (%)</th><th>Team</th><th>Roster</th></tr><tr><td class="tournament-results-place" rowspan="1"><img alt="PlacementIcon1" data-image-key="PlacementIcon1.png" data-image-name="PlacementIcon1.png" data-relevant="1" decoding="async" height="23" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/7/79/PlacementIcon1.png/revision/latest/scale-to-width-down/25?cb=20210522142824" width="25"/> 1</td><td class="tournament-results-prize" rowspan="1">$ 1,200,000</td><td rowspan="1">48%</td><td class="tournament-results-team"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe"><img alt="Atlanta FaZelogo std" data-image-key="Atlanta_FaZelogo_std.png" data-image-name="Atlanta FaZelogo std.png" dat

In [102]:
# Initialize lists to store the extracted data
cod_champs_placing = []
cod_champs_winnings = []
champs_percentage_prizepool = []
team_names = []
cod_champs_rosters = []

# Loop through the rows of the table to extract the data
for row in champs_element.find_all('tr')[1:]:  # Skipping the header row
    try:
        placing = row.find('td', class_='tournament-results-place').text.strip()
        cod_champs_placing.append(placing)
    except:
        cod_champs_placing.append('n/a')

    try:
        winnings = row.find('td', class_='tournament-results-prize').text.strip()
        cod_champs_winnings.append(winnings)
    except:
        cod_champs_winnings.append('n/a')

    try:
        percentage = row.find_all('td')[2].text.strip()  # Assuming the prize percentage is in the third column
        champs_percentage_prizepool.append(percentage)
    except:
        champs_percentage_prizepool.append('n/a')

    try:
        team_name_tag = row.find('span', class_='teamname')
        if team_name_tag:
            team_name = team_name_tag.text.strip()
            team_names.append(team_name)
        else:
            team_names.append('n/a')
    except:
        team_names.append('n/a')

    try:
        roster = row.find('td', class_='tournament-results-roster').text.strip()
        cod_champs_rosters.append(roster)
    except:
        cod_champs_rosters.append('n/a')

In [105]:
cod_champs_df = pd.DataFrame({
    'Cod_Champs_Place': cod_champs_placing,
    'Champs_Prize': cod_champs_winnings,
    'Prize (%)': champs_percentage_prizepool,
    'Team Name': team_names,
    'Champs_Roster': cod_champs_rosters
})
cod_champs_df

,Cod_Champs_Place,Champs_Prize,Prize (%),Team Name,Champs_Roster
0,1,"$ 1,200,000",48%,ATL FaZe,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ
1,2,"$ 650,000",26%,TOR Ultra,Cammy • CleanX • Bance • Insight • MarkyB
2,3,"$ 300,000",12%,DAL Empire,Shotzzy • iLLeY • C6 • Vivid • Rambo
3,4,"$ 150,000",6%,MIN RØKKR,Attach • Priestahh • MajorManiak • Standy • S...
4,5-6,"$ 75,000",3%,OpTic CHI,Scump • FormaL • Envoy • Dashy • Sender
5,n/a,n/a,n/a,NY Subliners,Mack • Asim • HyDra • Clayster • Revan
6,7-8,"$ 25,000",1%,FLA Mutineers,Owakening • Skyz • Neptune • Havok • Atura • ...
7,n/a,n/a,n/a,LA Thieves,Kenny • SlasheR • John • Drazah • JKap


In [106]:
# Update the n/a values
cod_champs_df.loc[cod_champs_df['Team Name'] == 'NY Subliners', ['Cod_Champs_Place', 'Champs_Prize', 'Prize (%)']] = ['5-6', '$ 75,000', '3%']
cod_champs_df.loc[cod_champs_df['Team Name'] == 'LA Thieves', ['Cod_Champs_Place', 'Champs_Prize', 'Prize (%)']] = ['7-8', '$ 25,000', '1%']
cod_champs_df.loc[cod_champs_df['Team Name'] == 'TOR Ultra', ['Cod_Champs_Place', 'Champs_Prize', 'Prize (%)']] = ['2', '$ 650,000', '26%']

In [107]:
cod_champs_df

,Cod_Champs_Place,Champs_Prize,Prize (%),Team Name,Champs_Roster
0,1,"$ 1,200,000",48%,ATL FaZe,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ
1,2,"$ 650,000",26%,TOR Ultra,Cammy • CleanX • Bance • Insight • MarkyB
2,3,"$ 300,000",12%,DAL Empire,Shotzzy • iLLeY • C6 • Vivid • Rambo
3,4,"$ 150,000",6%,MIN RØKKR,Attach • Priestahh • MajorManiak • Standy • S...
4,5-6,"$ 75,000",3%,OpTic CHI,Scump • FormaL • Envoy • Dashy • Sender
5,5-6,"$ 75,000",3%,NY Subliners,Mack • Asim • HyDra • Clayster • Revan
6,7-8,"$ 25,000",1%,FLA Mutineers,Owakening • Skyz • Neptune • Havok • Atura • ...
7,7-8,"$ 25,000",1%,LA Thieves,Kenny • SlasheR • John • Drazah • JKap


Building on top of the already existing information 
- series each team won and lost 
- map count as well 
- then to combine them all

In [30]:
champs_bracket = s_object.find('table',class_='wikitable md-table')
champs_bracket

<table class="wikitable md-table" id="md-table" style="min-width:22em"><tbody><tr><th colspan="9"><div style="position:relative;"><div class="table-header-showhide">[<span class="sections-toggler mdv-showbuttons" data-toggler-hide=".mdv-showbuttons" data-toggler-show=".mdv-allweeks">show all</span><span class="sections-toggler mdv-allweeks toggle-section-hidden" data-toggler-hide=".mdv-allweeks" data-toggler-show=".mdv-showbuttons">hide all</span>]</div></div>VODs &amp; Match Links</th></tr><tr><th colspan="9"><div style="position:relative;"><div class="table-header-showhide">[<span class="sections-toggler mdv-showbuttons mdv-showbutton1" data-toggler-hide=".mdv-showbutton1" data-toggler-show=".mdv-week1">show</span><span class="sections-toggler mdv-allweeks mdv-week1 toggle-section-hidden" data-toggler-hide=".mdv-week1" data-toggler-show=".mdv-showbutton1">hide</span>]</div></div>Winners Bracket</th></tr><tr class="column-label-small mdv-allweeks mdv-week1 toggle-section-hidden"><th>T

In [31]:
winners_champsbracket = champs_bracket.find_all('tr')[3:8]
winners_champsbracket

[<tr class="mdv-allweeks mdv-week1 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Dallas_Empire" data-to-titles="Dallas Empire||Tooltip:Dallas Empire" href="/wiki/Dallas_Empire" title="Dallas Empire"><img alt="Dallas Empirelogo std" data-image-key="Dallas_Empirelogo_std.png" data-image-name="Dallas Empirelogo std.png" data-relevant="0" decoding="async" height="15" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/c/c5/Dallas_Empirelogo_std.png/revision/latest/scale-to-width-down/36?cb=20191020015408" width="36"/></a>⁠</span><span class="teamname">DAL</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Minnesota_R_c3-_98-KKR" data-to-titles="Minnesota RØKKR||Tooltip:Minnesota RØKKR" href="/wiki/Minnesota_R%C3%98KKR" title="Minnesota RØKKR"><img al

In [110]:
# Initialize lists
team_names = []
series_counts = []
map_counts = []

# Loop through the tr elements
for row in winners_champsbracket:
    columns = row.find_all('td')

    # Extract team names
    team1_name = columns[0].find('span', class_='teamname').text.strip()
    team2_name = columns[1].find('span', class_='teamname').text.strip()

    # Extract series result
    result = columns[2].text.strip()
    team1_result, team2_result = map(int, result.split(' - '))

    # Determine series counts based on the result
    if team1_result == 3:
        team1_series_count = '1 - 0'
        team2_series_count = '0 - 1'
    elif team2_result == 3:
        team1_series_count = '0 - 1'
        team2_series_count = '1 - 0'
    else:
        team1_series_count = team2_series_count = 'n/a'

    # Append data to lists
    team_names.extend([team1_name, team2_name])
    series_counts.extend([team1_series_count, team2_series_count])
    map_counts.extend([f'{team1_result} - {team2_result}', f'{team2_result} - {team1_result}'])

In [111]:
# Print results to verify
print('Team Names:', team_names)
print('Series Counts:', series_counts)
print('Map Counts:', map_counts)

Team Names: ['DAL', 'MIN', 'CHI', 'NY', 'TOR', 'DAL', 'ATL', 'NY', 'ATL', 'DAL']
Series Counts: ['1 - 0', '0 - 1', '0 - 1', '1 - 0', '0 - 1', '1 - 0', '1 - 0', '0 - 1', '1 - 0', '0 - 1']
Map Counts: ['3 - 0', '0 - 3', '2 - 3', '3 - 2', '2 - 3', '3 - 2', '3 - 0', '0 - 3', '3 - 0', '0 - 3']


In [112]:
winners = pd.DataFrame({'Team Name':team_names, '(Champs/Lan - Series Counts)': series_counts, '(Champs/Lan - Map Counts)':map_counts
    
})
winners

,Team Name,(Champs/Lan - Series Counts),(Champs/Lan - Map Counts)
0,DAL,1 - 0,3 - 0
1,MIN,0 - 1,0 - 3
2,CHI,0 - 1,2 - 3
3,NY,1 - 0,3 - 2
4,TOR,0 - 1,2 - 3
5,DAL,1 - 0,3 - 2
6,ATL,1 - 0,3 - 0
7,NY,0 - 1,0 - 3
8,ATL,1 - 0,3 - 0
9,DAL,0 - 1,0 - 3


In [113]:
# Function to parse and sum counts from a string
def parse_and_sum(count_string):
    wins, losses = map(int, count_string.split(' - '))
    return wins, losses
#dealing with duplicates 


In [114]:
# Function to combine counts
def combine_counts(count1, count2):
    wins1, losses1 = parse_and_sum(count1)
    wins2, losses2 = parse_and_sum(count2)
    total_wins = wins1 + wins2
    total_losses = losses1 + losses2
    return f"{total_wins} - {total_losses}"


In [115]:
# Initialize a dictionary to store the combined data
combined_data = {}



In [116]:
# Loop through the existing DataFrame and combine the counts
for index, row in winners.iterrows():
    team_name = row['Team Name']
    series_count = row['(Champs/Lan - Series Counts)']
    map_count = row['(Champs/Lan - Map Counts)']
    
    if team_name in combined_data:
        combined_data[team_name]['(Champs/Lan - Series Counts)'] = combine_counts(combined_data[team_name]['(Champs/Lan - Series Counts)'], series_count)
        combined_data[team_name]['(Champs/Lan - Map Counts)'] = combine_counts(combined_data[team_name]['(Champs/Lan - Map Counts)'], map_count)
    else:
        combined_data[team_name] = {
            '(Champs/Lan - Series Counts)': series_count,
            '(Champs/Lan - Map Counts)': map_count
        }



In [117]:
# Create a new DataFrame from the combined data
winnersb_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
winnersb_df.rename(columns={'index': 'Team Name'}, inplace=True)
winnersb_df
#winners bracket dataframe and results 

,Team Name,(Champs/Lan - Series Counts),(Champs/Lan - Map Counts)
0,DAL,2 - 1,6 - 5
1,MIN,0 - 1,0 - 3
2,CHI,0 - 1,2 - 3
3,NY,1 - 1,3 - 5
4,TOR,0 - 1,2 - 3
5,ATL,2 - 0,6 - 0


In [32]:
#now getting loser bracket results 
losers_champsbracket = champs_bracket.find_all('tr')[10:16]
losers_champsbracket

[<tr class="mdv-allweeks mdv-week2 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Minnesota_R_c3-_98-KKR" data-to-titles="Minnesota RØKKR||Tooltip:Minnesota RØKKR" href="/wiki/Minnesota_R%C3%98KKR" title="Minnesota RØKKR"><img alt="Minnesota RØKKRlogo std" data-image-key="Minnesota_R%C3%98KKRlogo_std.png" data-image-name="Minnesota RØKKRlogo std.png" data-relevant="0" decoding="async" height="15" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/7/7f/Minnesota_R%C3%98KKRlogo_std.png/revision/latest/scale-to-width-down/36?cb=20191029151844" width="36"/></a>⁠</span><span class="teamname">MIN</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Los_Angeles_Thieves" data-to-titles="Los Angeles Thieves||Tooltip:Los Angeles Thieves" href="/wiki/Los_Ange

In [119]:
# Initialize lists
team_names = []
series_counts = []
map_counts = []

# Loop through the tr elements
for row in losers_champsbracket:
    columns = row.find_all('td')

    # Extract team names
    team1_name = columns[0].find('span', class_='teamname').text.strip()
    team2_name = columns[1].find('span', class_='teamname').text.strip()

    # Extract series result
    result = columns[2].text.strip()
    team1_result, team2_result = map(int, result.split(' - '))

    # Determine series counts based on the result
    if team1_result == 3:
        team1_series_count = '1 - 0'
        team2_series_count = '0 - 1'
    elif team2_result == 3:
        team1_series_count = '0 - 1'
        team2_series_count = '1 - 0'
    else:
        team1_series_count = team2_series_count = 'n/a'

    # Append data to lists
    team_names.extend([team1_name, team2_name])
    series_counts.extend([team1_series_count, team2_series_count])
    map_counts.extend([f'{team1_result} - {team2_result}', f'{team2_result} - {team1_result}'])

In [120]:
losers = pd.DataFrame({'Team Name':team_names, '(Champs/Lan - Series Counts)': series_counts, '(Champs/Lan - Map Counts)':map_counts
    
})
losers

,Team Name,(Champs/Lan - Series Counts),(Champs/Lan - Map Counts)
0,MIN,1 - 0,3 - 2
1,LAT,0 - 1,2 - 3
2,CHI,1 - 0,3 - 1
3,FLA,0 - 1,1 - 3
4,TOR,1 - 0,3 - 1
5,CHI,0 - 1,1 - 3
6,NY,0 - 1,1 - 3
7,MIN,1 - 0,3 - 1
8,TOR,1 - 0,3 - 2
9,MIN,0 - 1,2 - 3


In [121]:
# Initialize a dictionary to store the combined data
combined_data = {}

In [122]:
# Loop through the existing DataFrame and combine the counts
for index, row in losers.iterrows():
    team_name = row['Team Name']
    series_count = row['(Champs/Lan - Series Counts)']
    map_count = row['(Champs/Lan - Map Counts)']
    
    if team_name in combined_data:
        combined_data[team_name]['(Champs/Lan - Series Counts)'] = combine_counts(combined_data[team_name]['(Champs/Lan - Series Counts)'], series_count)
        combined_data[team_name]['(Champs/Lan - Map Counts)'] = combine_counts(combined_data[team_name]['(Champs/Lan - Map Counts)'], map_count)
    else:
        combined_data[team_name] = {
            '(Champs/Lan - Series Counts)': series_count,
            '(Champs/Lan - Map Counts)': map_count
        }

In [123]:
# Create a new DataFrame from the combined data
losersb_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
losersb_df.rename(columns={'index': 'Team Name'}, inplace=True)
losersb_df
#losers bracket dataframe and results 

,Team Name,(Champs/Lan - Series Counts),(Champs/Lan - Map Counts)
0,MIN,2 - 1,8 - 6
1,LAT,0 - 1,2 - 3
2,CHI,1 - 1,4 - 4
3,FLA,0 - 1,1 - 3
4,TOR,3 - 0,9 - 5
5,NY,0 - 1,1 - 3
6,DAL,0 - 1,2 - 3


In [124]:
#combineing both dataframes
# Combine the two DataFrames
combined_df = pd.concat([winnersb_df, losersb_df])

In [125]:
# Initialize a dictionary to store the aggregated results
team_stats = {}

In [126]:
# Aggregate the results
for index, row in combined_df.iterrows():
    team_name = row['Team Name']
    series_wins, series_losses = parse_and_sum(row['(Champs/Lan - Series Counts)'])
    map_wins, map_losses = parse_and_sum(row['(Champs/Lan - Map Counts)'])
    
    if team_name not in team_stats:
        team_stats[team_name] = {'Series Wins': 0, 'Series Losses': 0, 'Map Wins': 0, 'Map Losses': 0}
    
    team_stats[team_name]['Series Wins'] += series_wins
    team_stats[team_name]['Series Losses'] += series_losses
    team_stats[team_name]['Map Wins'] += map_wins
    team_stats[team_name]['Map Losses'] += map_losses

In [127]:
# Create a new DataFrame from the aggregated results
aggregated_data = {
    'Team Name': [],
    '(Champs/Lan - Series Counts)': [],
    '(Champs/Lan - Map Counts)': []
}

for team, stats in team_stats.items():
    series_count = f"{stats['Series Wins']} - {stats['Series Losses']}"
    map_count = f"{stats['Map Wins']} - {stats['Map Losses']}"
    aggregated_data['Team Name'].append(team)
    aggregated_data['(Champs/Lan - Series Counts)'].append(series_count)
    aggregated_data['(Champs/Lan - Map Counts)'].append(map_count)

final_df = pd.DataFrame(aggregated_data)

# Display the final DataFrame in a table-like format
final_df

,Team Name,(Champs/Lan - Series Counts),(Champs/Lan - Map Counts)
0,DAL,2 - 2,8 - 8
1,MIN,2 - 2,8 - 9
2,CHI,1 - 2,6 - 7
3,NY,1 - 2,4 - 8
4,TOR,3 - 1,11 - 8
5,ATL,2 - 0,6 - 0
6,LAT,0 - 1,2 - 3
7,FLA,0 - 1,1 - 3


In [128]:
#now getting grand finals results 
champs_gfbracket = champs_bracket.find_all('tr')[18]
champs_gfbracket

<tr class="mdv-allweeks mdv-week3 toggle-section-hidden"><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Atlanta_FaZe" data-to-titles="Atlanta FaZe||Tooltip:Atlanta FaZe" href="/wiki/Atlanta_FaZe" title="Atlanta FaZe"><img alt="Atlanta FaZelogo std" data-image-key="Atlanta_FaZelogo_std.png" data-image-name="Atlanta FaZelogo std.png" data-relevant="0" decoding="async" height="15" loading="lazy" src="https://static.wikia.nocookie.net/cod_esports_gamepedia_en/images/5/5b/Atlanta_FaZelogo_std.png/revision/latest/scale-to-width-down/36?cb=20191026200905" width="36"/></a>⁠</span><span class="teamname">ATL</span></span></td><td class="" rowspan="1"><span class="team"><span class="teamimage-right">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Toronto_Ultra" data-to-titles="Toronto Ultra||Tooltip:Toronto Ultra" href="/wiki/Toronto_Ultra" title="Toronto Ultra"><img alt="Toronto Ultralogo std" data-i

In [130]:
# Initialize lists to store the data
team_names = []
series_counts = []
map_counts = []

# Extract team names and results
teams = champs_gfbracket.find_all('span', class_='teamname')
results = champs_gfbracket.find_all('td')[2].text.strip()

team1 = teams[0].text.strip()
team2 = teams[1].text.strip()

team1_result, team2_result = map(int, results.split(' - '))

# Append data for team 1
team_names.append(team1)
if team1_result == 5:
    series_counts.append('1 - 0')
else:
    series_counts.append('0 - 1')
map_counts.append(f"{team1_result} - {team2_result}")

# Append data for team 2
team_names.append(team2)
if team2_result == 5:
    series_counts.append('1 - 0')
else:
    series_counts.append('0 - 1')
map_counts.append(f"{team2_result} - {team1_result}")

# Create DataFrame
grand_final_df = pd.DataFrame({
    'Team Name': team_names,
    '(Grand Final - Series Count)': series_counts,
    '(Grand Final - Map Count)': map_counts
})

grand_final_df

,Team Name,(Grand Final - Series Count),(Grand Final - Map Count)
0,ATL,1 - 0,5 - 3
1,TOR,0 - 1,3 - 5


In [131]:
# Create dictionaries for series and map counts aggregation
combined_data = {}

In [132]:
# Combine the existing combined_df data
for i, row in combined_df.iterrows():
    team = row['Team Name']
    series_count = row['(Champs/Lan - Series Counts)']
    map_count = row['(Champs/Lan - Map Counts)']
    
    if team not in combined_data:
        combined_data[team] = {'Series Count': series_count, 'Map Count': map_count}
    else:
        combined_data[team]['Series Count'] = combine_counts(combined_data[team]['Series Count'], series_count)
        combined_data[team]['Map Count'] = combine_counts(combined_data[team]['Map Count'], map_count)

In [133]:
# Add the grand final data
for i, row in grand_final_df.iterrows():
    team = row['Team Name']
    series_count = row['(Grand Final - Series Count)']
    map_count = row['(Grand Final - Map Count)']
    
    if team not in combined_data:
        combined_data[team] = {'Series Count': series_count, 'Map Count': map_count}
    else:
        combined_data[team]['Series Count'] = combine_counts(combined_data[team]['Series Count'], series_count)
        combined_data[team]['Map Count'] = combine_counts(combined_data[team]['Map Count'], map_count)


In [134]:
# Create a new DataFrame from the combined data
champs21_df = pd.DataFrame.from_dict(combined_data, orient='index').reset_index()
champs21_df.columns = ['Team Name', 'Champs - Series Count', 'Champs - Map Count']

# Display the final combined DataFrame
champs21_df

,Team Name,Champs - Series Count,Champs - Map Count
0,DAL,2 - 2,8 - 8
1,MIN,2 - 2,8 - 9
2,CHI,1 - 2,6 - 7
3,NY,1 - 2,4 - 8
4,TOR,3 - 2,14 - 13
5,ATL,3 - 0,11 - 3
6,LAT,0 - 1,2 - 3
7,FLA,0 - 1,1 - 3


Now need to combine all dataframes to get the information for the final of 2021 season 
- champs 21_df combine with cod_champs_df first then 
- pt distribution needs to be done before combining with combined_df
- pt distibution contains all pt dist and tournamanet placings
- cod_champs_df contains all cod champs rosters ,earngins, placings 
- combined_df contains all online series and lan series

In [135]:
new_team_names = [
    'Atlanta FaZe', 'Toronto Ultra', 'Dallas Empire','Minnesota RØKKR' ,'Optic Chicago',
    'New York Subliners', 'Florida Mutineers', 'Los Angeles Thieves',
]

cod_champs_df['Team Name'] = new_team_names
cod_champs_df

,Cod_Champs_Place,Champs_Prize,Prize (%),Team Name,Champs_Roster
0,1,"$ 1,200,000",48%,Atlanta FaZe,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ
1,2,"$ 650,000",26%,Toronto Ultra,Cammy • CleanX • Bance • Insight • MarkyB
2,3,"$ 300,000",12%,Dallas Empire,Shotzzy • iLLeY • C6 • Vivid • Rambo
3,4,"$ 150,000",6%,Minnesota RØKKR,Attach • Priestahh • MajorManiak • Standy • S...
4,5-6,"$ 75,000",3%,Optic Chicago,Scump • FormaL • Envoy • Dashy • Sender
5,5-6,"$ 75,000",3%,New York Subliners,Mack • Asim • HyDra • Clayster • Revan
6,7-8,"$ 25,000",1%,Florida Mutineers,Owakening • Skyz • Neptune • Havok • Atura • ...
7,7-8,"$ 25,000",1%,Los Angeles Thieves,Kenny • SlasheR • John • Drazah • JKap


In [136]:
new_team_names = [
    'Dallas Empire','Minnesota RØKKR' ,'Optic Chicago','New York Subliners','Toronto Ultra','Atlanta FaZe',  
     'Los Angeles Thieves','Florida Mutineers', 
]
champs21_df['Team Name'] = new_team_names
champs21_df


,Team Name,Champs - Series Count,Champs - Map Count
0,Dallas Empire,2 - 2,8 - 8
1,Minnesota RØKKR,2 - 2,8 - 9
2,Optic Chicago,1 - 2,6 - 7
3,New York Subliners,1 - 2,4 - 8
4,Toronto Ultra,3 - 2,14 - 13
5,Atlanta FaZe,3 - 0,11 - 3
6,Los Angeles Thieves,0 - 1,2 - 3
7,Florida Mutineers,0 - 1,1 - 3


In [137]:
# Merging the dataframes
combined_champsdf = pd.merge(cod_champs_df, champs21_df, on='Team Name')
combined_champsdf

,Cod_Champs_Place,Champs_Prize,Prize (%),Team Name,Champs_Roster,Champs - Series Count,Champs - Map Count
0,1,"$ 1,200,000",48%,Atlanta FaZe,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ,3 - 0,11 - 3
1,2,"$ 650,000",26%,Toronto Ultra,Cammy • CleanX • Bance • Insight • MarkyB,3 - 2,14 - 13
2,3,"$ 300,000",12%,Dallas Empire,Shotzzy • iLLeY • C6 • Vivid • Rambo,2 - 2,8 - 8
3,4,"$ 150,000",6%,Minnesota RØKKR,Attach • Priestahh • MajorManiak • Standy • S...,2 - 2,8 - 9
4,5-6,"$ 75,000",3%,Optic Chicago,Scump • FormaL • Envoy • Dashy • Sender,1 - 2,6 - 7
5,5-6,"$ 75,000",3%,New York Subliners,Mack • Asim • HyDra • Clayster • Revan,1 - 2,4 - 8
6,7-8,"$ 25,000",1%,Florida Mutineers,Owakening • Skyz • Neptune • Havok • Atura • ...,0 - 1,1 - 3
7,7-8,"$ 25,000",1%,Los Angeles Thieves,Kenny • SlasheR • John • Drazah • JKap,0 - 1,2 - 3


Now need to combine all dataframes to get the information for the final of 2021 season

- now combined_champsdf needs to got with pt_distribution
- then what ever name that is
- goes with combined_df

In [138]:
# Merge pt_distribution with combined_champs_df
finale_df = pd.merge(pt_distribution, combined_champsdf, on='Team Name', how='left')

# Fill missing values with 'NaN'
finale_df.fillna('NaN', inplace=True)
finale_df

#one dataframe already combined

,Team Name,Stage 1,Major 1,Stage 2,Major 2,Stage 3,Major 3,Stage 4,Major 4,Stage 5,...,5th - 6th Place,7th - 8th Place,9th - 10th Place,11th - 12th Place,Cod_Champs_Place,Champs_Prize,Prize (%),Champs_Roster,Champs - Series Count,Champs - Map Count
0,Atlanta FaZe,50,75,40,60,40,75,50,75,40,...,0,1,0,0,1,"$ 1,200,000",48%,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ,3 - 0,11 - 3
1,Toronto Ultra,10,20,20,75,50,50,40,50,40,...,0,1,0,0,2,"$ 650,000",26%,Cammy • CleanX • Bance • Insight • MarkyB,3 - 2,14 - 13
2,Dallas Empire,40,60,30,50,30,20,30,60,40,...,0,1,0,0,3,"$ 300,000",12%,Shotzzy • iLLeY • C6 • Vivid • Rambo,2 - 2,8 - 8
3,Optic Chicago,40,40,30,30,20,40,30,40,40,...,1,0,0,0,5-6,"$ 75,000",3%,Scump • FormaL • Envoy • Dashy • Sender,1 - 2,6 - 7
4,New York Subliners,30,50,40,20,40,60,40,30,10,...,1,2,0,0,5-6,"$ 75,000",3%,Mack • Asim • HyDra • Clayster • Revan,1 - 2,4 - 8
5,Minnesota RØKKR,30,10,30,40,20,10,30,30,30,...,1,0,2,0,4,"$ 150,000",6%,Attach • Priestahh • MajorManiak • Standy • S...,2 - 2,8 - 9
6,Los Angeles Thieves,40,30,10,30,30,30,10,0,30,...,4,0,0,1,7-8,"$ 25,000",1%,Kenny • SlasheR • John • Drazah • JKap,0 - 1,2 - 3
7,Florida Mutineers,10,20,20,10,30,30,30,20,30,...,1,2,2,0,7-8,"$ 25,000",1%,Owakening • Skyz • Neptune • Havok • Atura • ...,0 - 1,1 - 3
8,Seattle Surge,10,0,20,20,0,0,10,20,10,...,1,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN
9,London Royal Ravens,0,0,20,10,20,20,10,0,20,...,0,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
entire2021_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage),(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage),Total - Series Count,Total - Map Count,Total - Series Win Percentage,Total - Map Win Percentage
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0,12 - 4,75.0,50 - 29,63.0,34 - 7,122 - 53,83.0,70.0
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0,9 - 10,47.0,39 - 43,48.0,26 - 18,99 - 84,59.0,54.0
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0,5 - 6,45.0,22 - 21,51.0,17 - 19,69 - 70,47.0,50.0
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0,2 - 5,29.0,11 - 16,41.0,9 - 23,45 - 79,28.0,36.0
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0,3 - 6,33.0,13 - 21,38.0,8 - 26,44 - 87,24.0,34.0
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0,6 - 8,43.0,26 - 27,49.0,18 - 21,74 - 83,46.0,47.0
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0,6 - 6,50.0,26 - 26,50.0,20 - 17,75 - 73,54.0,51.0
7,New York Subliners,16 - 9,64.0,57 - 44,56.0,8 - 8,50.0,28 - 37,43.0,24 - 17,85 - 81,59.0,51.0
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0,10 - 9,53.0,41 - 36,53.0,26 - 18,97 - 74,59.0,57.0
9,Paris Legion,8 - 17,32.0,39 - 58,40.0,0 - 5,0.0,3 - 15,17.0,8 - 22,42 - 73,27.0,37.0


In [140]:
s2021_df = pd.merge(entire2021_df, finale_df, on='Team Name', how='outer')

# Display the combined DataFrame
s2021_df

,Team Name,(Stages/Online - Series Count),(Stages/Online - Series Win Percentage),(Stages/Online - Map Count),(Stages/Online - Map Win Percentage),(Stages/LAN - Series Counts),(Stages/LAN - Series Win Percentage),(Stages/LAN - Map Counts),(Stages/LAN - Map Win Percentage),Total - Series Count,...,5th - 6th Place,7th - 8th Place,9th - 10th Place,11th - 12th Place,Cod_Champs_Place,Champs_Prize,Prize (%),Champs_Roster,Champs - Series Count,Champs - Map Count
0,Atlanta FaZe,22 - 3,88.0,72 - 24,75.0,12 - 4,75.0,50 - 29,63.0,34 - 7,...,0.0,1.0,0.0,0.0,1,"$ 1,200,000",48%,aBeZy • Simp • Cellium • Arcitys • Crowder • RJ,3 - 0,11 - 3
1,Dallas Empire,17 - 8,68.0,60 - 41,59.0,9 - 10,47.0,39 - 43,48.0,26 - 18,...,0.0,1.0,0.0,0.0,3,"$ 300,000",12%,Shotzzy • iLLeY • C6 • Vivid • Rambo,2 - 2,8 - 8
2,Florida Mutineers,12 - 13,48.0,47 - 49,49.0,5 - 6,45.0,22 - 21,51.0,17 - 19,...,1.0,2.0,2.0,0.0,7-8,"$ 25,000",1%,Owakening • Skyz • Neptune • Havok • Atura • ...,0 - 1,1 - 3
3,London Royal Ravens,7 - 18,28.0,34 - 63,35.0,2 - 5,29.0,11 - 16,41.0,9 - 23,...,0.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Los Angeles Guerrillas,5 - 20,20.0,31 - 66,32.0,3 - 6,33.0,13 - 21,38.0,8 - 26,...,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,Los Angeles Thieves,12 - 13,48.0,48 - 56,46.0,6 - 8,43.0,26 - 27,49.0,18 - 21,...,4.0,0.0,0.0,1.0,7-8,"$ 25,000",1%,Kenny • SlasheR • John • Drazah • JKap,0 - 1,2 - 3
6,Minnesota RØKKR,14 - 11,56.0,49 - 47,51.0,6 - 6,50.0,26 - 26,50.0,20 - 17,...,1.0,0.0,2.0,0.0,4,"$ 150,000",6%,Attach • Priestahh • MajorManiak • Standy • S...,2 - 2,8 - 9
7,New York Subliners,16 - 9,64.0,57 - 44,56.0,8 - 8,50.0,28 - 37,43.0,24 - 17,...,1.0,2.0,0.0,0.0,5-6,"$ 75,000",3%,Mack • Asim • HyDra • Clayster • Revan,1 - 2,4 - 8
8,OpTic Chicago,16 - 9,64.0,56 - 38,60.0,10 - 9,53.0,41 - 36,53.0,26 - 18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Optic Chicago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,5-6,"$ 75,000",3%,Scump • FormaL • Envoy • Dashy • Sender,1 - 2,6 - 7


In [141]:
s2021_df.to_excel('Data - cdl_season_2021.xlsx', index=False)